# FixKit Example

In this example we demonstrate how to use FixKit to fix a bug in a Python function.
We will use the `middle_2` bug from the `tests4py` package as an example.

## Step 1: Getting the Faulty Program

We will first get the faulty program by checking out the `middle_2` bug from the `tests4py` package.

We are going to import the `tests4py` API as `t4p` and then call the `checkout` function with the `middle_2` bug and 
`tmp` as an output directory.

In [1]:
from pathlib import Path
import tests4py.api as t4p

tests4py :: INFO     :: Loading projects


In [2]:
report = t4p.checkout(t4p.middle_2, Path("tmp"))
if report.raised: raise report.raised

tests4py :: INFO     :: Copying https://github.com/smythi93/middle from /Users/marius/.t4p/projects/middle into tmp/middle_2... 
tests4py :: INFO     :: Resetting git at tmp/middle_2 to 029cb8beb7bfc0f2853dfa9504dcdfcc753b051e
tests4py :: INFO     :: Creating tmp location at /Users/marius/Desktop/work/projects/fixkit/tmp/tmp_middle
tests4py :: INFO     :: Copying required files to /Users/marius/Desktop/work/projects/fixkit/tmp/tmp_middle
tests4py :: INFO     :: Checkout buggy commit id eed99fa2741bd28744231dfcac0ea34679532bf9
tests4py :: INFO     :: Copying required files from /Users/marius/Desktop/work/projects/fixkit/tmp/tmp_middle
tests4py :: INFO     :: Create info file
tests4py :: INFO     :: Copying resources for middle_2


## Step 2: Setting up the Repair Approach

In this step, we will set up the repair approach.
We will use AE to fix the bug in `middle_2` in this example.

We will use the `PyAE` class from the `fixkit.repair.pyae` module to set up the repair approach.
We need to provide the source directory, the localization, and the maximal number of mutations `k`.
As the `middle_2` bug from `tests4py`, we will use the `Tests4PyLocalization` class from the 
`fixkit.localization.t4p` module to localize the fault.

Moreover, we will set the `is_t4p` parameter to `True` to indicate that the bug is from `tests4py`.

Finally, we will set the `line_mode` parameter to `True`. 
With this parameter, the repair approach will only consider statements that are inline, i.e., not containing a block.
For example, the repair approach will consider the statement `return x` but not the statement `if x: return x`.

In [3]:
from fixkit.repair.pyae import PyAE
from fixkit.localization.t4p import Tests4PyLocalization
from fixkit.constants import DEFAULT_EXCLUDES

In [4]:
approach = PyAE.from_source(
    Path("tmp", "middle_2"),
    excludes=DEFAULT_EXCLUDES,
    localization=Tests4PyLocalization(
        Path("tmp", "middle_2"),
        events=["line"],
        predicates=["line"],
        metric="Ochiai",
    ),
    k=1,
    is_t4p=True,
    line_mode=True,
)

fixkit :: INFO     :: Searching for statements in the source.
fixkit :: INFO     :: Building the initial candidate.


With the repair approach set up, we can now repair the bug in `middle_2`.

In [5]:
patches = approach.repair()

fixkit :: INFO     :: Localizing the faulty code locations.
tests4py :: INFO     :: Checking whether Tests4Py project
tests4py :: INFO     :: Loading projects
sflkit :: INFO     :: I found 10 events in /Users/marius/Desktop/work/projects/fixkit/tmp/middle_2/src/middle/__init__.py.
sflkit :: INFO     :: I found 10 events in /Users/marius/Desktop/work/projects/fixkit/tmp/middle_2.
tests4py :: INFO     :: Checking whether Tests4Py project
tests4py :: INFO     :: Loading projects
tests4py :: INFO     :: Checking for platform darwin
tests4py :: INFO     :: Check for activated python version
tests4py :: INFO     :: Using pyenv python 3.10.9
tests4py :: INFO     :: Activating virtual env
tests4py :: INFO     :: Run setup

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
tests4py :: INFO     :: Set compiled flag
tests4py :: INFO     :: Checking whether Tests4Py project
tests4py :: INFO     :: Loading projects
tests4py :: INFO     :: 

This will return a list of possible patches.
Since AE is deterministic and stops as soon as a possible repair is found, the list will contain only one patch, 
if one patch is found.

In [6]:
patch = patches[0]

We can now investigate the mutations the patch applies.

In [7]:
patch.mutations

[InsertBefore(1,3)]

Let us check what statements are affected by the patch.
The mutation inserts statement 3 before statement 1.

In [8]:
import ast
print(ast.unparse(patch.statements[3]))

return x


In [9]:
print(ast.unparse(patch.statements[1]))

return y


So the patch inserts the statement `return x` before the statement `return y`.

## Step 3: Applying the Patch

Finally, we can apply the patch to the faulty program to retrieve a patch file that can be used to fix the bug.

In [10]:
from fixkit.repair.patch import get_patch

In [11]:
print(get_patch(patch))

--- src/middle/__init__.py
+++ src/middle/__init__.py
@@ -3,10 +3,10 @@
         if x < y:
             return y
         elif x < z:
+            return x
             return y
-    else:
-        if x > y:
-            return y
-        elif x > z:
-            return x
-    return z
+    elif x > y:
+        return y
+    elif x > z:
+        return x
+    return z



Since the patch file is derived directly from the code instead of git, it may contain irrelevant artifacts.
In this case, the patch file removes code and replaces it with a semantically equivalent code.
This equivalent code is produced by the `ast` module that comprises the `else: if:` into an `elif:`.